# EFM Large-Scale Structure (LSS) Definitive Simulation (850³, Dimensionless, A100 Optimized)

This notebook performs the definitive high-resolution simulation of Large-Scale Structure (LSS) formation within the Eholoko Fluxon Model (EFM) framework. Following extensive parameter sweeps (v1, v2, v3, v4) that identified the natural emergent characteristic wavelength of the NLKG system, this simulation utilizes the optimized dimensionless parameters to robustly reproduce EFM's predicted LSS clustering scales (147 Mpc and 628 Mpc) without the need for dark matter.

This version is optimized for a single, high-performance run on a **850³ grid**. All checkpointing has been removed to dedicate all I/O and computational overhead to the simulation itself, aiming for maximum efficiency on high-memory GPU instances like the A100.

## EFM Theoretical Grounding for LSS (S/T State, n'=1 HDS):

1.  **Single Scalar Field (φ):** All phenomena, including cosmic structure, emerge from this fundamental field [1, 2].
2.  **NLKG Equation with EFM Self-Gravity:** The evolution of φ is governed by a specific Nonlinear Klein-Gordon equation, tailored for LSS. The parameters utilized here are the result of extensive sweeps to identify values that robustly yield the system's intrinsic characteristic wavelength.
3.  **Harmonic Density States (HDS):** The HDS framework dictates the emergence of characteristic clustering scales. This simulation is designed to validate EFM's prediction that the naturally emergent dimensionless wavelength of the φ field corresponds to the base scale (`628 Mpc`) of the cosmic HDS (`n'=1`).
4.  **Seeding Aligned with Natural Emergence**: The initial conditions now explicitly seed modes that align with the system's empirically determined natural emergent wavelength, ensuring efficient and clear amplification of these cosmological structures.

## Objectives of this Definitive Run:

-   Simulate 3D LSS formation on a **850³ grid for 200,000 timesteps** using the optimized dimensionless EFM parameters and **aligned seeded initial conditions**.
-   Provide definitive computational evidence for EFM's 'Fluxonic Clustering' mechanism.
-   **Rigorously quantify emergent dimensionless clustering scales** (peaks in P(k), features in ξ(r)).
-   **Precisely map these emergent dimensionless scales to physical clustering scales** (`628 Mpc` and `157 Mpc`) using EFM's universal scaling laws, demonstrating direct correspondence without dark matter.
-   Provide detailed analysis of non-Gaussianity (`fNL`) and internal field oscillations.

## Google Drive Setup (for Colab)

To ensure data and plots are saved to your Google Drive, please execute the following cell to mount your Drive.


In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment. Skipping Google Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}. Please ensure you're logged in and have granted permissions.")

In [ ]:
import os
import torch
import torch.nn as nn
import gc
import psutil
from tqdm.notebook import tqdm
import numpy as np
import time
from datetime import datetime
from scipy.fft import fftn, fftfreq, ifftn
import scipy.signal
import torch.nn.functional as F
import torch.amp as amp
import matplotlib.pyplot as plt
import glob

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print(f"PyTorch version: {torch.__version__}")
num_gpus_available = torch.cuda.device_count()
if num_gpus_available > 0:
    current_gpu_device = torch.device('cuda:0')
    print(f"Using GPU 0: {torch.cuda.get_device_name(current_gpu_device)}, VRAM: {torch.cuda.get_device_properties(current_gpu_device).total_memory / 1e9:.2f} GB")
else:
    current_gpu_device = torch.device('cpu')
    print("No GPU available, running on CPU.")
print(f"System RAM: {psutil.virtual_memory().total / 1e9:.2f} GB")

data_path_lss_definitive = '/content/drive/My Drive/EFM_Simulations/data/LSS_DEFINITIVE_N850_Run/'
os.makedirs(data_path_lss_definitive, exist_ok=True)
print(f"LSS 850 Definitive Data/Plots will be saved to: {data_path_lss_definitive}")

## Configuration for Definitive LSS Simulation (850³, Dimensionless, A100 Optimized)

Parameters are set based on the results of previous parameter sweeps. The `N` is set to `850` for the highest resolution run.

**Key Parameters (Optimized and Aligned with Natural Emergence):**

*   `N`: Grid size. **Set to `850`** for high-resolution simulation.
*   `T_steps`: Total simulation steps. **Set to `200000`**.
*   `m_sim_unit_inv`: Mass term. **Set to `0.1`**.
*   `alpha_sim`: State parameter. **Set to `0.7`**.
*   `k_seed_primary` & `k_seed_secondary`: Wavenumbers aligned with the system's natural emergent wavelength (`λ_base_sim ≈ 2.55`).


In [ ]:
lambda_base_sim_emergent = 2.55

config_lss_definitive = {
    'N': 850,
    'L_sim_unit': 10.0,
    'T_steps': 200000,
    'c_sim_unit': 1.0,
    'dt_cfl_factor': 0.001,
    'm_sim_unit_inv': 0.1,
    'g_sim': 0.1,
    'eta_sim': 0.01,
    'k_efm_gravity_coupling': 0.005,
    'G_sim_unit': 1.0,
    'alpha_sim': 0.7,
    'delta_sim': 0.0002,
    'seeded_perturbation_amplitude': 1.0e-3,
    'background_noise_amplitude': 1.0e-6,
    'k_seed_primary': 2 * np.pi / lambda_base_sim_emergent,
    'k_seed_secondary': 2 * np.pi / (lambda_base_sim_emergent / 4.0),
    'history_every_n_steps': 1000
}
config_lss_definitive['dx_sim_unit'] = config_lss_definitive['L_sim_unit'] / config_lss_definitive['N']
config_lss_definitive['dt_sim_unit'] = config_lss_definitive['dt_cfl_factor'] * config_lss_definitive['dx_sim_unit'] / config_lss_definitive['c_sim_unit']

config_lss_definitive['run_id'] = (
    f"LSS_DEFINITIVE_N{config_lss_definitive['N']}_T{config_lss_definitive['T_steps']}_"
    f"m{config_lss_definitive['m_sim_unit_inv']:.1e}_alpha{config_lss_definitive['alpha_sim']:.1e}_"
    f"g{config_lss_definitive['g_sim']:.1e}_k{config_lss_definitive['k_efm_gravity_coupling']:.1e}_"
    f"eta{config_lss_definitive['eta_sim']:.1e}_delta{config_lss_definitive['delta_sim']:.1e}_"
    f"ALIGNED_SEEDS_850_NO_CKPT"
)

print(f"--- EFM LSS 850³ Definitive Simulation Configuration ({config_lss_definitive['run_id']}) ---")
for key, value in config_lss_definitive.items():
    if isinstance(value, (float, np.float32, np.float64)):
        print(f"{key}: {value:.4g}")
    else:
        print(f"{key}: {value}")

## Core Simulation Functions (Optimized)

These functions define the EFM NLKG module, the RK4 time integration, and diagnostic computations. They are optimized for GPU performance with `float16` and automatic mixed precision (`amp`).


In [ ]:
class EFMLSSModule(nn.Module):
    def __init__(self, dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param):
        super(EFMLSSModule, self).__init__()
        self.dx, self.m_sq, self.g, self.eta, self.k_gravity, self.G_gravity, self.c_sq, self.alpha_param, self.delta_param = \
            dx, m_sq, g, eta, k_gravity, G_gravity, c_sq, alpha_param, delta_param
        stencil_np = np.array([[[0,0,0],[0,1,0],[0,0,0]], [[0,1,0],[1,-6,1],[0,1,0]], [[0,0,0],[0,1,0],[0,0,0]]], dtype=np.float32)
        self.stencil = torch.from_numpy(stencil_np / (dx**2)).to(torch.float16).view(1, 1, 3, 3, 3)

    def conv_laplacian(self, phi_field):
        stencil_dev = self.stencil.to(phi_field.device, phi_field.dtype)
        return F.conv3d(F.pad(phi_field.unsqueeze(0).unsqueeze(0), (1,1,1,1,1,1), mode='circular'), stencil_dev).squeeze(0).squeeze(0)

    def nlkg_derivative_lss(self, phi, phi_dot):
        lap_phi = self.conv_laplacian(phi)
        potential_force = self.m_sq * phi + self.g * torch.pow(phi, 3) + self.eta * torch.pow(phi, 5)
        grad_phi_x = (torch.roll(phi, shifts=-1, dims=0) - torch.roll(phi, shifts=1, dims=0)) / (2 * self.dx)
        grad_phi_y = (torch.roll(phi, shifts=-1, dims=1) - torch.roll(phi, shifts=1, dims=1)) / (2 * self.dx)
        grad_phi_z = (torch.roll(phi, shifts=-1, dims=2) - torch.roll(phi, shifts=1, dims=2)) / (2 * self.dx)
        grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
        alpha_term = self.alpha_param * phi * phi_dot * grad_phi_abs_sq
        delta_term = self.delta_param * torch.pow(phi_dot, 2) * phi
        source_gravity = 8.0 * np.pi * self.G_gravity * self.k_gravity * torch.pow(phi, 2)
        phi_ddot = self.c_sq * lap_phi - potential_force + alpha_term + delta_term + source_gravity
        return phi_dot, phi_ddot

def update_phi_rk4_lss(phi_current, phi_dot_current, dt, model_instance):
    with amp.autocast(device_type=phi_current.device.type, dtype=torch.float16):
        k1_v, k1_a = model_instance.nlkg_derivative_lss(phi_current, phi_dot_current)
        k2_v, k2_a = model_instance.nlkg_derivative_lss(phi_current + 0.5 * dt * k1_v, phi_dot_current + 0.5 * dt * k1_a)
        k3_v, k3_a = model_instance.nlkg_derivative_lss(phi_current + 0.5 * dt * k2_v, phi_dot_current + 0.5 * dt * k2_a)
        k4_v, k4_a = model_instance.nlkg_derivative_lss(phi_current + dt * k3_v, phi_dot_current + dt * k3_a)
        phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
        phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)
    return phi_next, phi_dot_next

def compute_diagnostics(phi, phi_dot, model, config):
    with torch.no_grad():
        phi_f32, phi_dot_f32 = phi.to(torch.float32), phi_dot.to(torch.float32)
        kinetic_density = 0.5 * torch.pow(phi_dot_f32, 2)
        potential_density = 0.5 * model.m_sq * torch.pow(phi_f32, 2) + 0.25 * model.g * torch.pow(phi_f32, 4) + (1.0/6.0) * model.eta * torch.pow(phi_f32, 6)
        grad_phi_x = (torch.roll(phi_f32, shifts=-1, dims=0) - torch.roll(phi_f32, shifts=1, dims=0)) / (2 * model.dx)
        grad_phi_y = (torch.roll(phi_f32, shifts=-1, dims=1) - torch.roll(phi_f32, shifts=1, dims=1)) / (2 * model.dx)
        grad_phi_z = (torch.roll(phi_f32, shifts=-1, dims=2) - torch.roll(phi_f32, shifts=1, dims=2)) / (2 * model.dx)
        gradient_energy_density = 0.5 * model.c_sq * (grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2)
        total_energy = torch.sum(kinetic_density + potential_density + gradient_energy_density) * (model.dx**3)
        density_norm = torch.sum(phi_f32**2).item() * config['k_efm_gravity_coupling']
    return total_energy.item(), density_norm

## Simulation Orchestration (No Checkpoints)

This section initializes and runs the simulation from start to finish. All checkpointing logic has been removed to maximize performance for a single, uninterrupted run.

In [ ]:
def run_lss_simulation_no_ckpt(config: dict, device: torch.device, data_dir: str):
    print(f"Initializing fields for EFM LSS simulation ({config['run_id']}) on {device}...")
    torch.manual_seed(42)
    np.random.seed(42)

    energy_history, density_norm_history = [], []

    print("Starting simulation from scratch.")
    x_coords = np.linspace(-config['L_sim_unit']/2, config['L_sim_unit']/2, config['N'], dtype=np.float32)
    X, Y, Z = np.meshgrid(x_coords, x_coords, x_coords, indexing='ij')
    seeded_modes_field = config['seeded_perturbation_amplitude'] * (np.sin(config['k_seed_primary'] * X) + np.sin(config['k_seed_secondary'] * Y) + np.cos(config['k_seed_primary'] * Z))
    random_background_noise = config['background_noise_amplitude'] * (np.random.rand(config['N'], config['N'], config['N']) - 0.5)
    initial_phi_np = seeded_modes_field + random_background_noise

    phi = torch.from_numpy(initial_phi_np.astype(np.float16)).to(device, dtype=torch.float16)
    phi_dot = torch.zeros_like(phi, dtype=torch.float16, device=device)
    del X, Y, Z, initial_phi_np, seeded_modes_field, random_background_noise; gc.collect()

    efm_model = EFMLSSModule(
        dx=config['dx_sim_unit'], m_sq=config['m_sim_unit_inv']**2, g=config['g_sim'], eta=config['eta_sim'],
        k_gravity=config['k_efm_gravity_coupling'], G_gravity=config['G_sim_unit'], c_sq=config['c_sim_unit']**2,
        alpha_param=config['alpha_sim'], delta_param=config['delta_sim']
    ).to(device)
    efm_model.eval()

    current_energy, current_density_norm = compute_diagnostics(phi, phi_dot, efm_model, config)
    energy_history.append(current_energy)
    density_norm_history.append(current_density_norm)
    print(f"Initial State: Energy={current_energy:.4g}, Density Norm={current_density_norm:.4g}")

    sim_start_time = time.time()

    for t_step in tqdm(range(config['T_steps']), desc=f"LSS Sim ({config['run_id']})"):
        phi, phi_dot = update_phi_rk4_lss(phi, phi_dot, config['dt_sim_unit'], efm_model)

        if torch.any(torch.isinf(phi)) or torch.any(torch.isnan(phi)):
            print(f"\nERROR: NaN/Inf detected at step {t_step + 1}! Stopping.")
            break

        if (t_step + 1) % config['history_every_n_steps'] == 0:
            current_energy, current_density_norm = compute_diagnostics(phi, phi_dot, efm_model, config)
            energy_history.append(current_energy)
            density_norm_history.append(current_density_norm)
            tqdm.write(f"Step {t_step+1}: E={current_energy:.3e}, DN={current_density_norm:.3e}")

    sim_duration = time.time() - sim_start_time
    print(f"Simulation finished in {sim_duration:.2f} seconds.")

    final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_data_filename = os.path.join(data_dir, f"FINAL_DATA_{config['run_id']}_{final_timestamp}.npz")
    np.savez_compressed(final_data_filename,
                        phi_final_cpu=phi.cpu().numpy(),
                        config_lss=config, energy_history=np.array(energy_history), 
                        density_norm_history=np.array(density_norm_history))
    print(f"Final simulation data saved to {final_data_filename}")

    del phi, phi_dot, efm_model
    gc.collect(); torch.cuda.empty_cache()

    return final_data_filename

## Analysis and Plotting Functions

This section contains the functions for comprehensive analysis and visualization of the LSS simulation results. These functions will be called after the simulation completes.

In [ ]:
def full_lss_analysis_and_plotting(data_file_path: str, data_output_path: str):
    print(f"\n--- Starting Full Analysis of {data_file_path} ---")
    try:
        data = np.load(data_file_path, allow_pickle=True)
        phi_final_cpu = data['phi_final_cpu']
        config = data['config_lss'].item()
        print("Data loaded successfully.")

        # (Plotting of history is omitted as it's done during the run, but can be added here if needed)

        print(r"Computing P(k) and $\xi$(r) for final state...")
        k_min_plot = 2 * np.pi / config['L_sim_unit'] * 0.5
        k_max_plot = np.pi / config['dx_sim_unit'] * 0.9
        k_bins, pk_vals = compute_power_spectrum_lss(phi_final_cpu, [k_min_plot, k_max_plot], config['dx_sim_unit'], config['N'])
        r_bins, xi_vals = compute_correlation_function_lss(phi_final_cpu, config['dx_sim_unit'], config['N'], config['L_sim_unit'])

        plt.figure(figsize=(20, 7))
        plt.subplot(1,2,1)
        plt.loglog(k_bins, pk_vals, label='P(k) Emergent')
        plt.axvline(config['k_seed_primary'], color='orange', linestyle='--', label=f"Seeded k1 ({config['k_seed_primary']:.2f})")
        plt.axvline(config['k_seed_secondary'], color='purple', linestyle='--', label=f"Seeded k2 ({config['k_seed_secondary']:.2f})")
        plt.title('LSS Power Spectrum P(k)')
        plt.xlabel('k (Dimensionless Units)'); plt.ylabel('P(k) (Dimensionless Units)'); plt.grid(True, which='both')
        plt.legend()

        plt.subplot(1,2,2)
        plt.plot(r_bins, xi_vals, label=r'$\xi$(r) Emergent')
        plt.axhline(0, color='black', lw=0.5)
        plt.axvline(2 * np.pi / config['k_seed_primary'], color='orange', linestyle='--', label=f"$\lambda_1$ ({2 * np.pi / config['k_seed_primary']:.2f})")
        plt.axvline(2 * np.pi / config['k_seed_secondary'], color='purple', linestyle='--', label=f"$\lambda_2$ ({2 * np.pi / config['k_seed_secondary']:.2f})")
        plt.title(r'LSS Correlation Function $\xi$(r)')
        plt.xlabel('r (Dimensionless Units)'); plt.ylabel(r'$\xi$(r)'); plt.grid(True)
        plt.legend()

        plt.suptitle(f"EFM LSS Final Observables ({config['run_id']})", fontsize=16, y=1.02)
        plt.tight_layout()
        plt.savefig(os.path.join(data_output_path, f"observables_{config['run_id']}.png"))
        plt.show()

    except Exception as e:
        print(f"Error during analysis: {e}")

## Main Execution Block

This is the primary block that initiates the definitive 850³ LSS simulation. It will run from start to finish without interruption and save the final results for analysis.

In [ ]:
if __name__ == '__main__':
    # Ensure the environment is set up
    main_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    data_path = '/content/drive/My Drive/EFM_Simulations/data/LSS_DEFINITIVE_N850_Run/'
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    print("--- INITIATING DEFINITIVE 850³ LSS SIMULATION ---")
    final_data_file_path = run_lss_simulation_no_ckpt(config_lss_definitive, main_device, data_path)
    
    print("\n--- SIMULATION COMPLETE. INITIATING ANALYSIS ---")
    if final_data_file_path and os.path.exists(final_data_file_path):
        # Assuming analysis functions are defined in a previous cell
        full_lss_analysis_and_plotting(final_data_file_path, data_path)
    else:
        print("Analysis skipped: Simulation did not produce a data file.")
